In [1]:
import pandas as pd

In [111]:
import json
import dateutil.parser
import http.client
import requests
import mysql.connector
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
class MySql:
    def __init__(self, user, password, database, host):
        self.user = user
        self.password = password
        self.database = database
        self.host = host

    def connection(self):
        connection = mysql.connector.connect(user=self.user, password=self.password, database=self.database,
                                             host=self.host)
        return connection

    @staticmethod
    def select(table, field, where_clause, connection_cursor):
        use_where = where_clause if where_clause != "" else ''

        query_select = "SELECT {} FROM {} {}".format(field, table, use_where)
        connection_cursor.execute(query_select)
        result = connection_cursor.fetchall()
        return result

    @staticmethod
    def insert(table, fields, list_to_insert, connection_cursor):
        values = '%s'
        values += (len(list_to_insert[0])-1)*', %s'
        use_fields = fields if fields != "" else ''
        connection_cursor.executemany("INSERT into {} {} VALUES ({})".format(table, use_fields, values),
                                      list_to_insert)



In [1]:
import json
import datetime
import collections
import requests


class InAccess:
    def __init__(self, raw_credentials, session):
        self.credentials = raw_credentials
        self.session = session

    def in_access_post(self):
        post_login = self.session.post('https://octopus.solarpark-online.com/ifms/login',
                                       data=self.credentials)
        return post_login

    @staticmethod
    def solar_get_parsed_measurements_15_m(measurement_id_value, session):
        date_yesterday = str((datetime.datetime.today() - datetime.timedelta(1)).strftime('%Y%m%d'))
        date_today = str(datetime.datetime.today().strftime('%Y%m%d'))
        returned_measurements = session.get('https://octopus.solarpark-online.com/ifms/sources/' +
                                            measurement_id_value + '/events?start_date=' + str(date_yesterday) +
                                            'T000000Z&end_date=' + str(date_today) + 'T000000Z')
        parsed_results = json.loads(returned_measurements.text)
        return parsed_results

    @staticmethod
    def solar_get_parsed_site_ids(siteLocationID, session):
        returned = session.get('https://octopus.solarpark-online.com/ifms/segments/' + siteLocationID + '/mlocs')
        parsed_results = json.loads(returned.text)
        return parsed_results

    @staticmethod
    def solar_get_measurement_ids(site_id, session):
        measurement_ids = session.get(
            'https://octopus.solarpark-online.com/ifms/mlocs/' + site_id)
        measurement_ids_result = json.loads(measurement_ids.text)
        return measurement_ids_result

    @staticmethod
    def wind_get_parser_measurements_10_m(owner, asset, signal, api_key):
        date_yesterday = (datetime.datetime.today() - datetime.timedelta(1)).strftime('%Y-%m-%d')
        date_today = datetime.datetime.today().strftime('%Y-%m-%d')
        response = requests.get('https://' + owner + '.breezeproduction.se/api/2.0/data.json?deviceIds=' + asset
                                + '&dataSignalIds=' + signal + '&timestampStart=' + date_yesterday +
                                'T00:10:00&timestampEnd=' + date_today +
                                'T00:00:00&resolution=10minute&aggregate=device&calculation=average&apiToken='
                                + api_key)

        parsed_measurements = json.loads(response.text, object_pairs_hook=collections.OrderedDict)
        return parsed_measurements

    @staticmethod
    def wind_get_site_details(owner,api_key):
        response = requests.get('https://' + owner + '.breezeproduction.se/api/2.0/devices.json?ApiToken=' + api_key
                                + '&fields=deviceId,title,site,turbineType.title,turbineType.model')

        results_wind_assets = json.loads(response.text)
        return results_wind_assets

    @staticmethod
    def wind_get_measurement_id_details(owner, device_id, api_key):
        response = requests.get('https://'+ owner +'.breezeproduction.se/api/2.0/datasignals.json?ApiToken='
                                + api_key
                                + '&fields=deviceId,title,site,turbineType.title,turbineType.model'
                                + '&deviceIds=' + device_id)

        result_measurement_id_details = json.loads(response.text)
        return result_measurement_id_details


MISSING INACCESS CREDENTIALS BELOW

In [159]:
dfs = []
i=1
with requests.session() as session:
    inaccess_connection = InAccess('{"username":"*****", "password": "****"}', session)
    api_login = inaccess_connection.in_access_post()

    if api_login.status_code == 200:
        mysql_connection = MySql('root', 'Password123!', 'energy',
                                 'localhost')
    connection = mysql_connection.connection()
    connection_cursor = connection.cursor()

#     data_from_solar_sites = mysql_connection.select("panelids", "panelId", "where siteId = '8748849a-b708-11e6-929d-42010afa015a' and namePanel in ('String 01-01-02', 'String 01-01-03')", connection_cursor)
    data_from_solar_sites = mysql_connection.select("panelids", "panelId", "where siteId = '819e65a0-4a58-11e5-aeb7-06876c02c9d4'", connection_cursor)
    
    for site in data_from_solar_sites:

        url = 'https://octopus.solarpark-online.com/ifms/sources/values?start_date=20180514T000000Z&end_date=20180515T00000000Z'
        

        for site_location_ID in site:
            list_of_dates = []
            list_of_measurements = []
            post_fields = "['"+site_location_ID+"']"
            request = session.post(url, data=post_fields)
            parsed_results = json.loads(request.content.decode())


            for result in parsed_results[0]['events']:
                if dateutil.parser.parse(result[0]).minute in (0, 15, 30, 45):

                    list_of_dates.append(dateutil.parser.parse(result[0]))
                    list_of_measurements.append(result[1])
            
            tempDF = pd.DataFrame()
            tempDF['timestamp'] = list_of_dates
            tempDF[site_location_ID] = list_of_measurements
#             print(tempDF)

            tempDF.set_index('timestamp', inplace=True)
            tempDF.index.tz = None
#             print(tempDF)
            dfs.append(tempDF)
            if i % 20 == 0:
                print(i)
            i +=1
masterDF = pd.concat(dfs, axis=1, join='outer')

#         data = pd.DataFrame({'index': list_of_dates, site[0]: list_of_measurements})
#         print(data)
#         pd.concat([df,data], axis=1)
#     print(df)
    

In [2]:
masterDF.fillna(0).astype(float).iloc[:,3:4].plot(figsize=(14,10))

NameError: name 'masterDF' is not defined

MISSING CREDENTIALS BELOW

In [164]:

#### TO LOOK FOR PARTICULAR PANELa USING THE ID

dfs = []
i=1
with requests.session() as session:
    inaccess_connection = InAccess('{"username":"****", "password": "****"}', session)
    api_login = inaccess_connection.in_access_post()

    if api_login.status_code == 200:
        mysql_connection = MySql('root', 'Password123!', 'energy',
                                 'localhost')
    connection = mysql_connection.connection()
    connection_cursor = connection.cursor()

#     data_from_solar_sites = mysql_connection.select("panelids", "panelId", "where siteId = '8748849a-b708-11e6-929d-42010afa015a' and namePanel in ('String 01-01-02', 'String 01-01-03')", connection_cursor)
#     data_from_solar_sites = mysql_connection.select("panelids", "panelId", "where siteId = '819e65a0-4a58-11e5-aeb7-06876c02c9d4'", connection_cursor)
    data_from_solar_sites = ['857a0d6e-4a58-11e5-aeb7-06876c02c9d4','857b7eb0-4a58-11e5-aeb7-06876c02c9d4','857ef090-4a58-11e5-aeb7-06876c02c9d4','85821536-4a58-11e5-aeb7-06876c02c9d4']
    for site_location_ID in data_from_solar_sites:

        url = 'https://octopus.solarpark-online.com/ifms/sources/values?start_date=20171205T000400Z&end_date=20171216T230000Z'
        

#         for site_location_ID in site:
        list_of_dates = []
        list_of_measurements = []
        post_fields = "['"+site_location_ID+"']"
        request = session.post(url, data=post_fields)
        parsed_results = json.loads(request.content.decode())


        for result in parsed_results[0]['events']:
            if dateutil.parser.parse(result[0]).minute in (0, 15, 30, 45):

                list_of_dates.append(dateutil.parser.parse(result[0]))
                list_of_measurements.append(result[1])

        tempDF = pd.DataFrame()
        tempDF['timestamp'] = list_of_dates
        tempDF[site_location_ID] = list_of_measurements
#             print(tempDF)

        tempDF.set_index('timestamp', inplace=True)
        tempDF.index.tz = None
#             print(tempDF)
        dfs.append(tempDF)
        if i % 20 == 0:
            print(i)
        i +=1
masterDF = pd.concat(dfs, axis=1, join='outer')

#         data = pd.DataFrame({'index': list_of_dates, site[0]: list_of_measurements})
#         print(data)
#         pd.concat([df,data], axis=1)
#     print(df)
    

In [161]:
masterDF.head(25)


,857a0d6e-4a58-11e5-aeb7-06876c02c9d4,857b7eb0-4a58-11e5-aeb7-06876c02c9d4,857ef090-4a58-11e5-aeb7-06876c02c9d4,85821536-4a58-11e5-aeb7-06876c02c9d4
timestamp,,,,
2017-12-05 00:15:18,0.0,0.0,1.08126835E-6,0.0
2017-12-05 00:30:18,1.89870182E-33,3.79113703E-6,1.01209022E-17,0.0
2017-12-05 00:45:18,8.96113762E-31,0.0,5.43374017E-27,2.0187328E-17
2017-12-05 01:00:18,2.0894384E-9,0.0,4.75255522E-20,0.0
2017-12-05 01:15:18,0.0,1.60316573E-12,1.89063414E-8,0.0
2017-12-05 01:30:18,0.0,0.0,0.0,0.0
2017-12-05 01:45:18,0.0,0.0,9.59336788E-9,2.3545392E-15
2017-12-05 02:00:18,3.34103275E-18,0.0,0.0,0.0
2017-12-05 02:15:18,1.7979116E-18,3.33684327E-14,2.14229403E-25,0.0


In [162]:
masterDF.to_csv('example1.csv')

In [26]:
df = pd.DataFrame()
data = pd.DataFrame({"A": range(3)})
df.append(data)

,A
0,0
1,1
2,2


In [27]:
data = pd.DataFrame({"A": range(3)})
df.append(data)

,A
0,0
1,1
2,2


In [29]:
data = pd.DataFrame({"B": range(3)})
df.append(data)

,B
0,0
1,1
2,2
